In [ ]:
from multitcn_components import TCNStack, DownsampleLayerWithAttention, LearningRateLogger
import tensorflow as tf
from tensorflow.keras.callbacks import ReduceLROnPlateau, LearningRateScheduler, EarlyStopping, ModelCheckpoint, CSVLogger
from sklearn import preprocessing
import numpy as np
import pandas as pd
from datetime import datetime
import tensorflow_addons as tfa
import uuid
import sys
from scipy.signal import correlate
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from tqdm import tqdm

In [ ]:
def windowed_dataset(series, time_series_number, window_size):
    """
    Returns a windowed dataset from a Pandas dataframe
    """
    available_examples= series.shape[0]-window_size + 1
    time_series_number = series.shape[1]
    inputs = np.zeros((available_examples,window_size,time_series_number))
    for i in range(available_examples):
        inputs[i,:,:] = series[i:i+window_size,:]
    return inputs 

def windowed_forecast(series, forecast_horizon):
    available_outputs = series.shape[0]- forecast_horizon + 1
    output_series_num = series.shape[1]
    output = np.zeros((available_outputs,forecast_horizon, output_series_num))
    for i in range(available_outputs):
        output[i,:]= series[i:i+forecast_horizon,:]
    return output

def shuffle_arrays_together(a,b):
    p = np.random.permutation(a.shape[0])
    return a[p],b[p]

def norm_cross_corr(a,b):
    nom = correlate(a,b)
    den = np.sqrt(np.sum(np.power(a,2))*np.sum(np.power(b,2)))
    return nom/den

def symm_mape(true,prediction):
    return 100*np.sum(2*np.abs(prediction-true)/(np.abs(true)+np.abs(prediction)))/true.size

def get_metrics(true,prediction,print_metrics=False):
        c = norm_cross_corr(true,prediction)
        extent = int((c.shape[0]-1)/2)
        max_corr_point = np.argmax(c)-extent
        max_corr = np.max(c)
        max_v = np.max(prediction)
        mse = mean_squared_error(true,prediction,squared=True)
        rmse = mean_squared_error(true,prediction,squared=False)
        mae = mean_absolute_error(true,prediction)
        r2 = r2_score(true,prediction)
        smape = symm_mape(true,prediction)
        if print_metrics:
            print("Max %f - Autocorr %d - MSE %f - RMSE %f - MAE %f - sMAPE %f%% - R^2 %f"%(max_v,max_corr_point,mse,rmse,mae,smape,r2))
        return [max_corr_point,mse,rmse,mae,smape,r2]

def get_confidence_interval_series(sample_array,confidence_level=0.95):
    bounds = stats.t.interval(confidence_level,sample_array.shape[0]-1)
    samples_mean = np.mean(sample_array,axis=0)
    samples_std = np.std(sample_array,axis=0,ddof=1)
    lower_bound = samples_mean + bounds[0]*samples_std/np.sqrt(sample_array.shape[0])
    upper_bound = samples_mean + bounds[1]*samples_std/np.sqrt(sample_array.shape[0])
    return samples_mean, lower_bound, upper_bound

def present_mean_metrics(metrics):
    print("Autocorr\t\t MSE\t\t RMSE\t\t MAE\t\t sMAPE\t\t R^2")
    print("%10.4f\t %10.4f\t %10.4f\t %10.4f\t %10.4f\t %10.4f"% tuple(np.mean(metrics,axis=0)))
    print("+-",)
    print("%10.4f\t %10.4f\t %10.4f\t %10.4f\t %10.4f\t %10.4f"% tuple(np.std(metrics,axis=0,ddof=1)))

In [ ]:
loss = 'mse'
#Dataset parameters
window_length = 96
forecast_horizon = 24
preprocessor = preprocessing.MinMaxScaler()
out_preprocessor = preprocessing.MinMaxScaler()
# preprocessor = preprocessing.StandardScaler(with_mean=0,with_std=1)
# out_preprocessor = preprocessing.StandardScaler(with_mean=0,with_std=1)
shuffle_train_set = True
scale_output = True
training_percentage = 0.75
experiment_target = F"Forecasting,{forecast_horizon} steps ahead"
experiment_complete = False

In [ ]:
############## Set up model ##########################
class MTCNAModel(tf.keras.Model):
    
    def __init__(self,tcn_layer_num,tcn_kernel_size,tcn_filter_num,window_size,forecast_horizon, use_bias, kernel_initializer, tcn_dropout_rate,tcn_dropout_format,tcn_activation, tcn_final_activation, tcn_final_stack_activation):
        super(MTCNAModel, self).__init__()
        self.window_size = window_size
        self.tcn_filter_num = tcn_filter_num
        

        #Create stack of TCN layers    
        self.lower_tcn = TCNStack(tcn_layer_num,tcn_filter_num,tcn_kernel_size,window_size,use_bias,kernel_initializer,tcn_dropout_rate,tcn_dropout_format,tcn_activation,tcn_final_activation,tcn_final_stack_activation)
    
        #Create stack of dense layers
        self.co_seq = tf.keras.models.Sequential()
        self.co_seq.add(tf.keras.layers.Flatten())
        self.co_seq.add(tf.keras.layers.Dense(forecast_horizon,activation=None))
        
        #Create stack of dense layers
        self.nox_seq = tf.keras.models.Sequential()
        self.nox_seq.add(tf.keras.layers.Flatten())
        self.nox_seq.add(tf.keras.layers.Dense(forecast_horizon,activation=None))
        
        #Create stack of dense layers
        self.no2_seq = tf.keras.models.Sequential()
        self.no2_seq.add(tf.keras.layers.Flatten())
        self.no2_seq.add(tf.keras.layers.Dense(forecast_horizon,activation=None))
        
        #Create stack of dense layers
        self.benzene_seq = tf.keras.models.Sequential()
        self.benzene_seq.add(tf.keras.layers.Flatten())
        self.benzene_seq.add(tf.keras.layers.Dense(forecast_horizon,activation=None))
        
    def call(self, input_tensor):
        x = self.lower_tcn(input_tensor)
        co = self.co_seq(x)
        nox = self.nox_seq(x)
        no2 = self.no2_seq(x)
        benzene = self.benzene_seq(x)
        return co,benzene,nox,no2

In [ ]:
################ Prepare dataset ###########################

### Note details for logging purposes
dataset_description = "Italian air quality data"
dataset_preprocessing = """Drop time information, Remove NAN rows at end, Replace missing values with 0"""

data = pd.read_csv("Datasets/AirQualityUCI.csv",sep=';',decimal=',')
## Remove NaN rows
data = data.drop(np.arange(9357,9471,1))
# Remove emtpy columns
data = data.drop(['Unnamed: 15','Unnamed: 16'],axis=1)


#Create date object for easy splitting according to dates
dateobj = pd.to_datetime(data["Date"],dayfirst=True) + pd.to_timedelta(data["Time"].str.replace(".00.00",":00:00"))

### For now remove timestamp and output values
data = data.drop(columns=["Date","Time"],axis=1)

#Drop column due to high number of missing values
data = data.drop(['NMHC(GT)'],axis=1)

# Replace missing values with 0
data = data.replace(-200,0)

# Reorganize columns in preparation for second stage (first columns are in order of outputs)
columns = ['CO(GT)','C6H6(GT)','NOx(GT)','NO2(GT)','PT08.S1(CO)','PT08.S2(NMHC)','PT08.S3(NOx)','PT08.S4(NO2)','PT08.S5(O3)','T','RH','AH']
data = data[columns]


## Add date object for splitting
data['DateObj'] = dateobj

In [ ]:
#Split data based on dates
training_start_date = pd.Timestamp(year=2004,month=3,day=10)

# Preceding values used only for creating final graph and predicting first values of test set
holdout_preceding_date = pd.Timestamp(year=2004, month=11, day=11)


holdout_set_start_date = pd.Timestamp(year=2004, month=12, day=11)
holdout_set_end_date = pd.Timestamp(year=2005, month=4, day=5)

training_data = data.loc[(data['DateObj']>=training_start_date) & (data['DateObj'] < holdout_set_start_date)]
test_data = data.loc[(data['DateObj'] >= holdout_set_start_date) & (data['DateObj'] < holdout_set_end_date)]

pre_evaluation_period = data.loc[(data['DateObj'] >= holdout_preceding_date) & (data['DateObj'] < holdout_set_start_date)]

input_variables = list(training_data.columns)

training_data = training_data.drop(['DateObj'],axis=1)
test_data = test_data.drop(['DateObj'],axis=1)

In [ ]:
##Select prediction target
targets = ['CO(GT)','C6H6(GT)','NOx(GT)','NO2(GT)',]
labels = np.array(training_data[targets])


if scale_output:
    out_preprocessor.fit(labels)
    if "Normalizer" in str(out_preprocessor.__class__):
        ## Save norm so in case of normalizer we can scale the predictions correctly
        out_norm = np.linalg.norm(labels)
        labels = preprocessing.normalize(labels,axis=0)
    else:
        labels= out_preprocessor.transform(labels)


num_input_time_series = training_data.shape[1]


### Make sure data are np arrays in case we skip preprocessing
training_data = np.array(training_data)

### Fit preprocessor to training data
preprocessor.fit(training_data)

if "Normalizer" in str(preprocessor.__class__):
    ## Save norm so in case of normalizer we can scale the test_data correctly
    in_norm = np.linalg.norm(training_data,axis=0)
    training_data = preprocessing.normalize(training_data,axis=0)
else:
    training_data = preprocessor.transform(training_data)

In [ ]:

### Create windows for all data
data_windows = windowed_dataset(training_data[:-forecast_horizon],num_input_time_series,window_length)
label_windows = windowed_forecast(labels[window_length:],forecast_horizon)

### Transpose outputs to agree with model output
label_windows = np.transpose(label_windows,[0,2,1])


samples = data_windows.shape[0]


## Shuffle windows
if shuffle_train_set:
    data_windows, label_windows = shuffle_arrays_together(data_windows,label_windows)

### Create train and validation sets
train_x = data_windows
train_y = [label_windows[:,i,:] for i in range(len(targets))]


## In order to use all days of test set for prediction, append training window from preceding period
pre_test_train = pre_evaluation_period[test_data.columns][-window_length:]
test_data = pd.concat([pre_test_train,test_data])

## Create windowed test set with same process
test_labels = np.array(test_data[targets])

#### Preprocess data
test_data = np.array(test_data)

if "Normalizer" in str(preprocessor.__class__):
    test_data = test_data/in_norm
else:
    test_data = preprocessor.transform(test_data)

test_x = windowed_dataset(test_data[:-forecast_horizon],num_input_time_series,window_length)
test_y = np.transpose(windowed_forecast(test_labels[window_length:],forecast_horizon),[0,2,1])

## Create pre test period for visualization
pre_test_target = np.append(np.array(pre_evaluation_period[targets]),test_labels[:window_length])

total_samples = train_x.shape[0] + test_x.shape[0]

In [ ]:
##################### Initialize model parameters ########################
## For simplicity all time series TCNs have the same parameters, though it is relatively easy to change this
tcn_kernel_size = 3
tcn_layer_num = 5
tcn_use_bias = True
tcn_filter_num = 128
tcn_kernel_initializer = 'random_normal'
tcn_dropout_rate = 0.3 
tcn_dropout_format = "channel"
tcn_activation = 'relu'
tcn_final_activation = 'linear'
tcn_final_stack_activation = 'relu'
loss = [loss]*len(targets)

In [ ]:
# ### Check for GPU


## Make only given GPU visible   
gpus = tf.config.experimental.list_physical_devices('GPU')
mirrored_strategy = None

print("GPUs Available: ", gpus)
if len(gpus)==0:
    device = "CPU:0"
else:
    print("Enter number of gpus to use:")
    gpu_num = input()
    if len(gpu_num)!=0 and gpu_num.isdigit():
        gpu_num = int(gpu_num)
    if gpu_num==1:
        print("Enter index of GPU to use:")
        gpu_idx = input()
        if len(gpu_idx)!=0 and gpu_idx.isdigit():
            gpu_idx = int(gpu_idx)
        tf.config.experimental.set_visible_devices(gpus[gpu_idx], 'GPU')
        device = "GPU:0"
    else:
        mirrored_strategy = tf.distribute.MirroredStrategy(devices=[F"GPU:{i}" for i in range(gpu_num)])
        device = " ".join([F"GPU:{i}" for i in range(gpu_num)])

In [ ]:
### Set evaluation seed to affect dropout random execution
print("Enter a seed for the evaluation:")
seed = input()
if len(seed)!=0 and seed.isdigit():
    seed = int(seed)
else:
    seed = 192
np.random.seed(seed)
tf.random.set_seed(seed)

In [ ]:
test_x_days.shape

In [ ]:
### ## Set up test model
## From all the test samples keep individual, non overlapping days
test_x_days = test_x[0::forecast_horizon,:]
true_y = np.transpose(test_y[0::forecast_horizon,:],(0,2,1)).reshape((-1,len(targets)))

test_dropout = 0.3

with tf.device(device):
    test_model = MTCNAModel(tcn_layer_num,tcn_kernel_size,tcn_filter_num,window_length,forecast_horizon, tcn_use_bias, tcn_kernel_initializer, test_dropout, tcn_dropout_format, tcn_activation, tcn_final_activation, tcn_final_stack_activation)
_ = test_model(test_x_days[0:1])


    
from os import listdir
weight_names = listdir("BaselineWeights-AirQ/")
print(weight_names)
dropout_runs_per_weight = 20


metrics_number = 6
samples_per_prediction = dropout_runs_per_weight*len(weight_names)

## Enable dropout
tf.keras.backend.set_learning_phase(1)
errors  = np.zeros((samples_per_prediction,test_x_days.shape[0]*forecast_horizon,len(targets)))
predictions = np.zeros((samples_per_prediction,test_x_days.shape[0]*forecast_horizon,len(targets)))
metrics = np.zeros((samples_per_prediction,metrics_number,len(targets)))

for i in tqdm(range(len(weight_names))):
    test_model.load_weights("BaselineWeights-AirQ/"+weight_names[i])
    for j in range(dropout_runs_per_weight):
        cur_pred = np.asarray(test_model(test_x_days)).reshape((len(targets),-1)).T
        if scale_output and "Normalizer" in str(out_preprocessor.__class__):
            cur_pred *= (out_norm)
        else:
            cur_pred = out_preprocessor.inverse_transform(cur_pred)
        predictions[i*dropout_runs_per_weight+j,:] = cur_pred
        errors[i*dropout_runs_per_weight+j,:] = cur_pred - true_y
        for target in range(len(targets)):
            metrics[i*dropout_runs_per_weight+j,:,target] = np.asarray(get_metrics(true_y[:,target],cur_pred[:,target],print_metrics=False))
        
        
        

In [ ]:
sns.set()
for var_idx in range(len(targets)):
    print(targets[var_idx])
    present_mean_metrics(metrics[...,var_idx])
    
    fig = plt.figure(figsize=(20,10))
    plt.hist((predictions[...,var_idx]-np.median(predictions[...,var_idx],axis=0)).flatten(),bins=20,alpha=0.5,range=(-2,2),label="Dropout predictiomn - median")
    plt.hist(errors[...,var_idx].flatten(),alpha=0.5,range=(-5,5),bins=20,label="Prediction - Actual")
    plt.legend()
    plt.show()

In [ ]:
confidence_level = 0.95
pred_mean, lower_bound,upper_bound = get_confidence_interval_series(predictions,confidence_level)

In [ ]:
test_x_days.shape

In [ ]:


preceding_points = 24
from_day =50
to_day = 51

pred_plot_range = range(preceding_points,preceding_points+(to_day-from_day)*forecast_horizon)
pred_plot_range1 = range(preceding_points-24,preceding_points+(to_day-from_day)*forecast_horizon-24)
pred_sp = from_day*forecast_horizon
pred_ep = to_day*forecast_horizon

for i in range(len(targets)):
    fig = plt.figure(figsize=(20,10))
    plt.plot(pred_plot_range,pred_mean[pred_sp:pred_ep,i],marker="o",label="Prediction")
    plt.fill_between(pred_plot_range, lower_bound[pred_sp:pred_ep,i], upper_bound[pred_sp:pred_ep,i], alpha=0.3)
    
    if from_day==0:
        plt.plot(pre_test_target[-preceding_points:,i],label="Pretest period", marker="o")
    else:
        plt.plot(true_y[pred_sp-preceding_points:pred_sp,i],label="Pretest period", marker="o")
    plt.plot(pred_plot_range,true_y[from_day*forecast_horizon:to_day*forecast_horizon,i],marker="o",label="True data")

    plt.grid(True)
    plt.legend()
#    plt.xticks()
    plt.show()